In [1]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
np.set_printoptions(precision=2, linewidth=140)

path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

In [2]:
n, m = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

In [3]:
# num hidden
nh = 50

In [4]:
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

In [5]:
def lin(x, w, b):
    return x@w + b

In [6]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [7]:
def relu(x):
    return x.clamp_min(0.)

In [8]:
t = relu(t)
t

tensor([[16.71,  0.00,  6.27,  ...,  8.47,  0.00,  0.00],
        [18.79,  0.00,  4.62,  ..., 10.59,  6.40,  0.00],
        [11.95,  1.91,  7.92,  ...,  3.96,  1.43,  0.00],
        ...,
        [ 0.00,  0.94,  3.64,  ..., 16.63,  8.00,  0.00],
        [ 2.63,  0.00, 12.83,  ...,  5.43,  4.04,  0.00],
        [ 2.10,  0.00,  9.26,  ...,  0.00,  9.66,  6.48]])

In [9]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [10]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

In [11]:
res.shape, y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [12]:
(res-y_valid).shape #not what we want

torch.Size([10000, 10000])

In [13]:
res[:,0].shape

torch.Size([10000])

In [14]:
res.squeeze().shape

torch.Size([10000])

In [15]:
y_train, y_valid = y_train.float(), y_valid.float()
preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [16]:
def mse(output,targ):
    return (output[:,0]-targ).pow(2).mean()

In [17]:
mse(preds, y_train)

tensor(1081.51)

 ## Gradients

In [18]:
from sympy import symbols, diff
x,y = symbols('x y')
diff(x**2,x)

2*x

In [19]:
diff(3*x**2+9,x)

6*x

In [36]:
def lin_grad(inp, out, w, b):
    #grad of matmul with respect to input
    import pdb; pdb.set_trace()
    inp.g = out.g @ w.t()
    w.g = inp.T@out.g
    b.g = out.g.sum(0)

In [42]:
def forward_and_backward(inp, targ):
    #forward pass
    #import pdb; pdb.set_trace()
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    loss = diff.pow(2).mean()
    import pdb; pdb.set_trace()
    #backward pass
    out.g = 2.*diff[:,None]/inp.shape[0] #dloss/dout
    lin_grad(l2, out, w2, b2) 
    l1.g = (l1>0).float() * l2.g 
    lin_grad(inp,l1,w1,b1)

In [43]:
forward_and_backward(x_train, y_train)

> /tmp/ipykernel_602491/111450975.py(11)forward_and_backward()
      9     import pdb; pdb.set_trace()
     10     #backward pass
---> 11     out.g = 2.*diff[:,None]/inp.shape[0] #dloss/dout
     12     lin_grad(l2, out, w2, b2)
     13     l1.g = (l1>0).float() * l2.g

ipdb> n
> /tmp/ipykernel_602491/111450975.py(12)forward_and_backward()
     10     #backward pass
     11     out.g = 2.*diff[:,None]/inp.shape[0] #dloss/dout
---> 12     lin_grad(l2, out, w2, b2)
     13     l1.g = (l1>0).float() * l2.g
     14     lin_grad(inp,l1,w1,b1)

ipdb> out.shape
torch.Size([50000, 1])
ipdb> targ.shape
torch.Size([50000])
ipdb> q


In [35]:
#x_train.g.zero_()

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [24]:
x_train.g

tensor([[    -0.02,      0.01,      0.00,  ...,      0.00,     -0.00,      0.01],
        [    -0.00,      0.00,      0.00,  ...,      0.00,     -0.00,      0.00],
        [    -0.01,      0.01,      0.01,  ...,      0.00,     -0.00,      0.00],
        ...,
        [    -0.00,     -0.00,     -0.00,  ...,     -0.00,     -0.00,     -0.00],
        [    -0.00,      0.01,      0.01,  ...,      0.01,     -0.01,      0.00],
        [     0.00,     -0.00,     -0.00,  ...,     -0.00,      0.00,     -0.00]])

In [25]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

## pytorch autograd

In [27]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [28]:
def forward(inp, targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [30]:
loss = forward(xt2,y_train)
loss.backward()

In [31]:
from fastcore.test import test_close

In [32]:
test_close(w22.grad, w2g, eps=0.01)
test_close(b22.grad, b2g, eps=0.01)
test_close(w12.grad, w1g, eps=0.01)
test_close(b12.grad, b1g, eps=0.01)
test_close(xt2.grad, ig , eps=0.01)

# REfactor model

In [39]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp > 0).float() * self.out.g

In [41]:
class Lin():
    def __init__(self, w, b):
        self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [78]:
class Mse():
    def __call__(self, inp, targ):
        self.inp , self.targ = inp, targ
        self.out = mse(inp, targ)
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0] #solo para un nodo de salida?
        

In [79]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x,targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

In [80]:
model = Model(w1, b1, w2, b2)

In [81]:
loss = model(x_train, y_train)

In [82]:
model.backward()

In [83]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

## module.forward

In [86]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out  
    def forward(self):
        raise Exception('Not implemented')
    def backward(self):
        self.bwd(self.out, *self.args)
    def bwd(self):
        raise Exception('Not implemented')

In [104]:
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.)
    def bwd(self, out, inp):
        inp.g = (inp > 0).float() * out.g

In [109]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b
    def forward(self, inp):
        return inp@self.w + self.b
    def bwd(self,out,inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [110]:
class Mse(Module):
    def forward(self, inp, targ):
        return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ):
        inp.g = 2 * (inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]

In [111]:
model = Model(w1, b1, w2, b2)

In [112]:
loss = model(x_train, y_train)

In [113]:
model.backward()

In [114]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

## Autograd

In [115]:
from torch import nn
import torch.nn.functional as F

In [120]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in, n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp):
        return inp@self.w + self.b

In [129]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh, n_out)]
    def forward(self, x, targ):
        for l in self.layers:
            x = l(x)
        return F.mse_loss(x, targ[:,None])

In [130]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [131]:
l0 = model.layers[0]
l0.b.grad

tensor([   -17.21,      7.03,    101.29,     74.58,     -4.47,     63.51,     59.04,    193.68,      2.44,    -21.20,    101.63,      2.69,
            81.84,     67.91,    133.15,    -47.66,    215.08,    -57.13,    -89.19,     24.66,     99.81,     61.46,    198.97,     55.98,
           -86.37,    -30.38,    -28.61,      2.21,    -68.43,      0.11,     21.68,      0.22,      9.14,    109.53,      2.54,      9.45,
            22.46,     61.57,     60.60,      9.37,     11.73,    -11.93,     22.53,     51.65,   -101.33,    -62.67,    -17.56,     -2.98,
            39.20,    104.26])